# Optimizing SMACross Trading with MultiModal Agent

In this demo, we introduce an agent workflow that write trading strategy and refine it throught backtesting using the **BackTrader** library.

In [ ]:
import os
import autogen
from autogen.cache import Cache

from finrobot.functional.quantitative import BackTraderUtils
from finrobot.toolkits import register_toolkits, register_code_writing
from textwrap import dedent

In [ ]:
config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-0125-preview"],
    },
)
llm_config = {
    "config_list": config_list,
    "timeout": 120,
    "temperature": 0
}

from finrobot.functional.coding import default_path

# Intermediate strategy modules will be saved in this directory
work_dir = default_path
os.makedirs(work_dir, exist_ok=True)

For this task, we need:
- A normal llm agent as data provider: Call charting functions and provide instructions for multimodal agent
- A multimodal agent as market analyst: Extract the necessary information from the chart and analyze the future trend of this stock.
- A user proxy to execute python functions and control the conversations.

In [ ]:
strategist = autogen.AssistantAgent(
    name="Trade_Strategist",
    system_message=dedent("""
        You are a trading strategist known for your expertise in developing sophisticated trading algorithms. 
        Your task is to utilize the existing SmaCross strategy (for which you don't need to write code, just specifying params) 
        or leverage your coding skills to create a customized trading strategy using the BackTrader Python library, and save it as a Python module. 
        You must provide trading strategies based on user demands and optimize them according to backtesting results.
        However, if backtesting came back with failed attempts, you should help with the examination and fix bugs.
        Reply TERMINATE to executer when the strategy is ready to be tested.
        """),
    llm_config=llm_config,
)
strategist_executor = autogen.UserProxyAgent(
    name="Trade_Strategist_Executor",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": work_dir,
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
register_code_writing(strategist, strategist_executor)

analyst = autogen.AssistantAgent(
    name="Backtesting_Analyst",
    system_message=dedent("""
        You are a backtesting analyst with a strong command of quantitative analysis tools. 
        Your primary role is to employ backtesting tools to rigorously evaluate the efficacy of designated trading strategies. 
        Upon completing your analysis, you are responsible for compiling and reporting the results in a clear and comprehensive manner, providing key insights that will inform further strategy development and refinement.
        However, if not all necessary information needed for backtesting is provided, you can skip backtesting and ask the strategist for additional details.
        Reply TERMINATE when the backtesting and analysis is complete.
        """),
    llm_config=llm_config,
)
analyst_executor = autogen.UserProxyAgent(
    name="Backtesting_Analyst_Executor",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": work_dir,
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
register_toolkits([BackTraderUtils.back_test], analyst, analyst_executor)


In [ ]:
def reflection_message_analyst(recipient, messages, sender, config):
    print("Reflecting strategist's response ...")
    last_msg = recipient.chat_messages_for_summary(sender)[-1]['content']
    return "Message from Trade Strategist is as follows:" + last_msg + "\n\nIf the strategist has completed his work properly, he should have designated a predefined trading strategy or saved a strategy he defined as a Python module. " \
        "Based on his information, conduct a backtest on the specified stock and strategy, and report your backtesting results back to the strategist."

def reflection_message_strategist(recipient, messages, sender, config):
    print("Reflecting analyst's response ...")
    last_msg = recipient.chat_messages_for_summary(sender)[-1]['content']
    return "Message from Backtesting Analyst is as follows:" + last_msg + "\n\nIf the analyst has completed his work properly, he should have completed the backtesting for the specified strategy and reported the results. " \
        "Based on the backtesting results, assess whether the current strategy meets the requirements. If the current strategy still does not meet the requirements, or if you believe that other strategies need to be tested, " \
        "continue to refine or provide new strategies, and then resubmit your backtesting requirements to the analyst."

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=False         # User Proxy dont need to execute code here
)
user_proxy.register_nested_chats(
    [
        {
            "sender": analyst_executor,
            "recipient": analyst,
            "message": reflection_message_analyst,
            "max_turns": 2,
            "summary_method": "last_msg",
        }
    ],
    trigger=[strategist, strategist_executor],
)
user_proxy.register_nested_chats(
    [
        {
            "sender": strategist_executor,
            "recipient": strategist,
            "message": reflection_message_strategist,
            "max_turns": 5,
            "summary_method": "last_msg",
        }
    ],
    trigger=[analyst, analyst_executor],
)

To prevent unexpected chat sequence arrangements in group chats, we opt for manual orchestration for this task. After the data provider supplies the data, the user proxy summarizes it and then presents it to the Multimodal market analyst for analysis.

In [ ]:
company = "Microsoft"
start_date = "2023-01-01"
end_date = "2024-01-01"

task = dedent(f"""
    Based on {company}'s stock data from {start_date} to {end_date}, develop a trading strategy that performs well on this stock. 
    You can start your research by providing an SMACross strategy as your baseline, then after receiving the backtest analysis, you might adjust parameters to try again 
    or create a more optimized trade by writing your own python strategy class with BackTrader's `Strategy`. 
    Once you have determined your strategy, you need to inform the backtesting analyst which strategy to use for backtesting on what data. 
    If it is your own written strategy class, you need to report the corresponding module path and class name, as well as any parameters that the strategy might require.
""")

with Cache.disk() as cache:
    user_proxy.initiate_chat(
        recipient=strategist,
        message=task,
        max_turns=10,
        summary_method="last_msg"
    )